In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

c:\Users\ASUS\anaconda3\envs\atlas\lib\site-packages\pydantic\_internal\_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
960,"Solid comedy entertainment, with musical inter...",negative
282,i love bad shark movies. i really do. i laugh ...,negative
251,If you want to be cynical and pedantic you cou...,positive
946,1st watched 5/17/2002 - 3 out of 10(Dir-Ewald ...,negative
465,This BBC series is excellent. I am no Paleonto...,positive


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
df = normalize_text(df)
df.head()

,review,sentiment
960,solid comedy entertainment musical interlude g...,negative
282,love bad shark movie really do laugh hysterica...,negative
251,want cynical pedantic could point opening raf ...,positive
946,st watched dir ewald andre dupont fairly lame ...,negative
465,bbc series excellent paleontologist series giv...,positive


In [5]:
df['sentiment'].value_counts()

sentiment
negative    263
positive    237
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
960,solid comedy entertainment musical interlude g...,0
282,love bad shark movie really do laugh hysterica...,0
251,want cynical pedantic could point opening raf ...,1
946,st watched dir ewald andre dupont fairly lame ...,0
465,bbc series excellent paleontologist series giv...,1


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [15]:
vectorizer = CountVectorizer(max_features=50)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']


In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/sumityesade14/capstone-proj.mlflow')
dagshub.init(repo_owner='sumityesade14', repo_name='capstone-proj', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Accessing as sumityesade14

Initialized MLflow to track repo "sumityesade14/capstone-proj"

Repository sumityesade14/capstone-proj initialized!

<Experiment: artifact_location='mlflow-artifacts:/6cb47ddfe5ef4fb993c608b1e97367e8', creation_time=1753938433336, experiment_id='0', last_update_time=1753938433336, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [20]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.2)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-08-01 11:56:01,749 - INFO - Starting MLflow run...
2025-08-01 11:56:02,381 - INFO - Logging preprocessing parameters...
2025-08-01 11:56:03,832 - INFO - Initializing Logistic Regression model...
2025-08-01 11:56:03,841 - INFO - Fitting the model...
2025-08-01 11:56:03,909 - INFO - Model training complete.
2025-08-01 11:56:03,909 - INFO - Logging model parameters...
2025-08-01 11:56:04,327 - INFO - Making predictions...
2025-08-01 11:56:04,331 - INFO - Calculating evaluation metrics...
2025-08-01 11:56:04,342 - INFO - Logging evaluation metrics...
2025-08-01 11:56:06,374 - INFO - Saving and logging the model...
c:\Users\ASUS\anaconda3\envs\atlas\lib\site-packages\_distutils_hack\__init__.py:15: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the tradition

In [13]:
import mlflow
print(mlflow.__version__)


2.8.1


In [14]:
! pip install mlflow==2.8.1

In [ ]:
import sys
print(sys.executable)


c:\Users\ASUS\anaconda3\envs\atlas\python.exe


In [ ]:
! pip uninstall -y mlflow

Found existing installation: mlflow 2.8.1
Uninstalling mlflow-2.8.1:
  Successfully uninstalled mlflow-2.8.1


In [4]:
pip show mlflow

Name: mlflow
Version: 2.8.1
Summary: MLflow: A Platform for ML Development and Productionization
Home-page: https://mlflow.org/
Author: Databricks
Author-email: 
License: Apache License 2.0
Location: c:\users\asus\anaconda3\envs\atlas\lib\site-packages
Requires: alembic, click, cloudpickle, databricks-cli, docker, entrypoints, Flask, gitpython, importlib-metadata, Jinja2, markdown, matplotlib, numpy, packaging, pandas, protobuf, pyarrow, pytz, pyyaml, querystring-parser, requests, scikit-learn, scipy, sqlalchemy, sqlparse, waitress
Required-by: 
Note: you may need to restart the kernel to use updated packages.
